# Introduction to Human In The Loop in Agent Framework

Workflows in the Microsoft Agent Framework allow developers to design, monitor, and control how multiple AI agents and logic components interact to complete complex tasks.

One of the ways to bring in control and manage agent workflows is to add human approvals. When agents require any user input, for example to approve a function call, this is referred to as a **human-in-the-loop** pattern. 

### Learning Objectives

By the end of this notebook, you will be able to:

- Understand how user approvals work in Agent Framework
- Create AI functions that require human approval using `approval_mode`
- Work with different ChatMessage content types (Text, FunctionCall, FunctionResult, Approval)
- Build complete workflows with approval gates
- Access and inspect conversation threads for observaibility

Before we begin, let's first understand how Agent Framework handles external system interactions:

### User Approval Overview

In AF, agent execution is fundamentally about the **data flow** - a workflow will keep running as long as there is data being passed. Agents don't "call" each other or pass control in a traditional procedural sense. Instead, they communicate through typed messages flowing between executors through edges.

The content of the message can also be used to let the application approve or disapprove some actions the agent wants to take.
**User Approval Content** type enables HITL approvals within the standard message flow.

The approval mechanism intercepts the normal tool invocation flow, pausing execution until human approval is granted or denied:

<img src="images/reqresp.png" alt="User Approval Content Type" style="width:70%; height:40%; margin:20px auto">

When a ChatAgent needs to invoke a Tool that requires approval:

1. **Initial Inference**: The ChatAgent uses the Model to make an inference and determines it needs to invoke the Tool.

2. **Approval Request**: Before executing, the Tool sends a request back to the ChatAgent, which surfaces to the Application as `FunctionApprovalRequestContent` (a special ChatMessage content type).

3. **User Response**: The Application presents this approval request to the user, who responds (Yes/No). This response is sent back to the ChatAgent as `FunctionApprovalResponseContent`.

4. **Tool Execution**: Once the ChatAgent receives approval, the Tool executes and returns its result to the ChatAgent.

5. **Final Inference**: The Model makes another inference using the Tool's result to generate a natural language response as `TextContent`.

6. **Complete Response**: The Application receives both the `FunctionResultContent` (the raw tool output) and the `TextContent` (the Model's interpretation of that result).



---
### Setup

First, let's validate our environment and create the Azure OpenAI client.

In [ ]:
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env

validate_env()

In [ ]:
import os
from agent_framework.azure import AzureOpenAIChatClient

chat_client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
)

print("Client created successfully!")

### Example 1: Simple Approval Flow

We'll create an email agent with two functions:
- `send_email_notification` - executes automatically
- `send_bulk_email` - requires approval

**1. Define functions with approval modes**

Use the `@ai_function` decorator to specify metadata. To create a function that requires approval, you can use the `approval_mode` parameter so the agent can pause and ask for user approval:

In [ ]:
from typing import Annotated
from agent_framework import ai_function, ChatMessage, Role, ChatAgent 

@ai_function
def send_email_notification(recipient: Annotated[str, "Email address"]) -> str:
    """Send a simple notification email."""
    return f"Notification sent to {recipient}"

@ai_function(approval_mode="always_require")
def send_bulk_email(recipient_list: Annotated[str, "Comma-separated email addresses"]) -> str:
    """Send email to multiple recipients. Requires approval."""
    recipients = recipient_list.split(",")
    return f"Bulk email sent to {len(recipients)} recipients: {recipient_list}"

Now let's create an agent and run it with a query that will trigger the approval-required function:

In [ ]:
agent = ChatAgent(
    chat_client=chat_client,
    name="EmailAgent",
    instructions="You are an email assistant. Use bulk email for multiple recipients.",
    tools=[send_email_notification, send_bulk_email],
)

result = await agent.run("Send an email to alice@example.com, bob@example.com, and charlie@example.com")

**2. First Agent Run and User Input Request:**

This agent run `result` is an `AgentRunResponse` object which has a `user_input_requests` attribute of type `list[UserInputRequestContents]`. The run returns one `UserInputRequestContent` type message per tool call, and often there are several approval requests per agent run so it's important to inspect all pending approvals:

```python
    if result.user_input_requests:
        print("Approval required:")
        for user_input_needed in result.user_input_requests:
            print(f"Function: {user_input_needed.function_call.name}")

```

Since you now have a function that requires approval, the agent can respond with a request for approval instead of executing the function directly and returning the result.

**3. Create the Approval Response:**

Use `.create_response(approved=True/False)` on the approval request to create a response, then wrap it in a `ChatMessage` with `Role.USER`. This links your decision back to the original function call request.

**4. Resuming Execution:**

Resume the agent by providing the full conversation history: the original query, the agent's function call request (as ASSISTANT), and your approval response (as USER). This reconstructs the conversation flow so the agent can continue.

See the implementation of the pattern below:

In [ ]:
if result.user_input_requests:
    print("Approval required:")
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")
        
    # Simulate approval
    user_approval = False

    # Create the approval response
    approval_message = ChatMessage(
        role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )

    final_result = await agent.run([
        "Send an email to team@company.com, sales@company.com, and support@company.com",
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])

print(final_result.text)

> *Note*: When we don't have a thread, we need to pass the full conversation history in the second agent run - the query, all requests and their associated responses, so we're not actually 'resuming' the first run but starting from the beginning (but now we can proceed with tool execution since we have collected the approvals)

This pattern ensures human oversight while giving your application full control over how approval requests are presented to users.

### Example 2 - Handling Approvals in a Loop

The following example registers two tools (`get_user` and `submit_user_data`) and maintains a running conversation throughout the session. When the agent needs to call functions requiring approval, it returns all approval requests. We then collect user input for each pending function call and send all approval responses back in a single follow-up agent run.

The framework automatically:
- Retrieves the full conversation context from the thread
- Matches approvals to pending function calls by `call_id`
- Executes approved functions and continues the conversation

This means you only send new information (the approvals) on each run while the framework handles state management behind the scenes.

First, let's define some simple tools. In reality, these would typically interact with actual databases and external APIs:

In [ ]:
from agent_framework import ai_function
from typing import Annotated


@ai_function(approval_mode="always_require")
def submit_user_data(
    name: Annotated[str, "User's full name"],
    role: Annotated[str, "User's role or department"],
    user_id: Annotated[int, "User's ID number"],
) -> str:
    """
    Function that submits user data to the system. 
    This function requires approval.
    When the user asks to submit data, call this function.
    """
    
    return (
        f"User data for '{name}' (ID: {user_id}, role: {role}) has been submitted "
        f"to the system."
    )


@ai_function(
    description="Retrieves the current user's information"
)
def get_user() -> dict:
    """
    Returns a dict with user information including name, role, and ID.
    This function does not require approval.
    """
    # Static user data return
    return {
        "name": "John Doe",
        "role": "Engineering Manager",
        "user_id": 1042
    }

With our tools defined, we can now implement the main interaction loop that handles user inputs.

Run the cell and try:
- `"Get user 123"` - No approval needed
- `"Create user Charlie with role Developer id 789"` - Requires approval
- Try approving and rejecting to see different behaviors

In [ ]:
import json
from agent_framework import ChatAgent, ChatMessage, Role
from agent_framework import (
    FunctionApprovalRequestContent
)

# Initialize the agent with tools and instructions
agent = ChatAgent(
    chat_client=chat_client,
    name="UserAgent",
    instructions=(
        "You are an agent for user management operations." 
        "You assist with submitting and retrieving current user information."
        "For submitting user data, you need: name, role, and user ID."
    ),
    tools=[get_user, submit_user_data],
)

thread = agent.get_new_thread()

print("User Management Agent - Handling Approvals in a Loop")
print("Type 'quit' or 'q' to exit.\n")

while True:

    # Get user input
    user_input = input("You: ").strip()
    if user_input.lower() in ("quit", "q"):
        print(" Finished the session")
        break
    if not user_input:
        continue

    # First agent run: process user query
    run = await agent.run(user_input, thread=thread)

    # Inspect the agent's response
    print("AGENT RUN DETAILS:")
    print(f"   Response ID: {result.response_id}")
    print(f"   User Query: '{user_input}'")
    print(f"   Messages in response: {len(result.messages)}")

    for i, msg in enumerate(result.messages, 1):
        print(f"\n   Message {i} - Role: {msg.role}")
        for content in msg.contents:
            if isinstance(content, FunctionApprovalRequestContent):
                parsed_args = content.function_call.parse_arguments()
                args_pretty = json.dumps(parsed_args, indent=2)
                print(f"      [Approval Request]:")
                print(f"         Function: {content.function_call.name}")
                print(f"         Arguments:\n{args_pretty}")
                print(f"         Call ID: {content.function_call.call_id}")
                print(f"         ID: {content.id}")


    # Check if the agent needs approval for any function calls
    if run.user_input_requests:
        print("\n\nAPPROVALS REQUIRED:")
        approval_messages: list[ChatMessage] = []

        # Iterate through each approval request and collect user decisions
        for request in run.user_input_requests:
            print(f"- Approval needed for: {request.function_call.name}")
            print(f"  Arguments: {request.function_call.arguments}")
            print(f"  Function call ID: {request.function_call.call_id}")

            # Get user's approval decision
            approved = input(f"Approve '{request.function_call.name}'? (yes/no): ").strip().lower() == "yes"

            # Add the user's approval response 
            approval_messages.append(
                ChatMessage(role=Role.USER, contents=[request.create_response(approved)])
            )

        # Second agent run: submit approvals and continue execution
        followup = await agent.run(approval_messages, thread=thread)

        print("\nAgent:", followup.text)

    else:
        # No approvals needed - display the response directly
        print(run.text)
    
    print()

In the first agent run, we pass the user's query along with the thread that persists only for the duration of the console session:
```python
    thread = agent.get_new_thread()
    result = await agent.run(user_input, thread=thread)
```

Threads maintain conversation history automatically, eliminating the need for manual context reconstruction. The framework treats each subsequent run as a continuation of the same conversation and uses the `call_id` to link each approval response to its corresponding function call request.

>*Note:* In the second agent run, we only need to pass the user approval message and the same thread as in the first run so every turn references the same context.

### Recap - ChatMessage Content Types
Agents communicate with applications via different message content types (`TextContent`, `FunctionCallContent`, `FunctionApprovalRequestContent`, etc.). Messages are represented by the `ChatMessage` class and all content type classes inherit from the base `BaseContent` class.

Understanding the different content types in ChatMessages is crucial for working with approvals. Here are the main types to help you identify what you can extract from each:

| Content Type              | Description                                                    | Key Properties                                           |
|---------------------------|----------------------------------------------------------------|----------------------------------------------------------|
| TextContent               | Textual content that can be both input and output from the agent | `text`                                                   |
| FunctionCallContent       | A request by an AI service to invoke a function tool          | `name`, `arguments`, `call_id`, `exception`                           |
| FunctionResultContent     | The result returned from a function tool invocation             | `result`, `call_id`, `exception`                                        |
| ErrorContent              | Error information when processing fails                    | `message`, `error_code`, `details`                             |
| UsageContent              | Token usage and billing information from the AI service.               | `details` |

For a detailed description of all available content types and their use, refer to the [`agent_framework` library docs](https://learn.microsoft.com/en-us/python/api/agent-framework-core/agent_framework?view=agent-framework-python-latest) and [AF docs](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/running-agents?pivots=programming-language-python)


Let's use the conversation history stored in the thread to understand what happened during the conversation and what information is retrievable from the messages:

In [ ]:
from agent_framework import (
    TextContent,
    FunctionCallContent,
    FunctionResultContent
)

messages = await thread.message_store.list_messages()

print("\n=== FULL THREAD HISTORY ===")
for i, msg in enumerate(messages):
    print(f"\n[Message {i+1}] Role: {msg.role}")
    
    for j, content in enumerate(msg.contents):
        if isinstance(content, TextContent):
            print(f"   Text Content:")
            print("      ", content.text)
        
        elif isinstance(content, FunctionCallContent):
            print(f"   Function Call Content:")
            print(f"      Name: {content.name}")
            print(f"      Arguments: {content.arguments}")
            print(f"      Call ID: {content.call_id}")

        elif isinstance(content, FunctionResultContent): 
            print(f"   Function Result Content:")
            print("      ", content.result)
            print(f"      Call ID: {content.call_id}")
        
        else:
            print(f"   Unknown content type: {type(content)}")

### Exercise - Simple Workflow with Approval Tools

In the next exercise, you'll build an agent that helps users manage their finances while requiring human approval for sensitive operations like money transfers and bill payments. This pattern is critical for production systems where certain actions need oversight before execution.

#### Step 1 - Define Your Data Model and AI Functions

First, let's set up the data structure for user queries and the AI functions that our agent can call.

*Note*: These functions use mock data for demonstration purposes and are purely for side effect generation. In a production system, they would connect to real banking APIs and databases. 

In [ ]:
from dataclasses import dataclass
from agent_framework import executor, AgentExecutorResponse, WorkflowContext, WorkflowBuilder, FunctionApprovalResponseContent, Executor, handler
from typing import Never

@dataclass
class FinanceQuery:
    user: str
    request: str

@ai_function
def get_account_balance(account_type: str) -> dict:
    """Get the current balance for an account."""
    accounts = {
        "checking": {"balance": 5240.50, "currency": "USD"},
        "savings": {"balance": 15780.25, "currency": "USD"},
        "credit": {"balance": -1205.80, "currency": "USD"},
    }
    return accounts.get(account_type, {"error": "Account not found"})


@ai_function
def get_recent_transactions(account_type: str, limit: int = 5) -> list[dict]:
    """Get recent transactions for an account."""
    transactions = {
        "checking": [
            {"date": "2025-11-06", "description": "Grocery Store", "amount": -125.40},
            {"date": "2025-11-05", "description": "Salary Deposit", "amount": 3500.00},
            {"date": "2025-11-03", "description": "Electric Bill", "amount": -89.50},
        ],
        "credit": [
            {"date": "2025-11-06", "description": "Online Shopping", "amount": -245.30},
            {"date": "2025-11-04", "description": "Restaurant", "amount": -67.80},
        ],
    }
    return transactions.get(account_type, [])[:limit]


@ai_function(approval_mode="always_require")
async def transfer_money(
    from_account: str,
    to_account: str,
    amount: float,
) -> str:
    """Transfer money between accounts."""
    
    return f"Successfully transferred ${amount:.2f} from {from_account} to {to_account}"


@ai_function(approval_mode="always_require")
async def pay_bill(
    account: str,
    payee: str,
    amount: float,
) -> str:
    """Pay a bill from an account."""
    
    return f"Successfully paid ${amount:.2f} to {payee} from {account}"

Then implement the following:
- `QueryRouter` which validates and forwards the request
- `conclude_workflow` which extracts and outputs the final agent response:

In [ ]:
class QueryRouter(Executor):
    def __init__(self) -> None:
        super().__init__(id="query_router")

    @handler
    async def route(self, query: FinanceQuery, ctx: WorkflowContext[str]) -> None:
        """Route the finance query to the agent."""
        
        # TODO: Surface user request messages
        # TODO: Send the message to the next executor


@executor(id="conclude_workflow")
async def conclude_workflow(
    response: AgentExecutorResponse,
    ctx: WorkflowContext[Never, str],
) -> None:
    """Output the final response."""

    # TODO: Extract the text and conclude


#### Step 2 - Build the Workflow and Implement the Approval Loop
Now let's assemble all the parts into a complete workflow and implement the human-in-the-loop approval mechanism. In the following section you will need to build the workflow and handle response requests yourself:

In [ ]:
def create_finance_agent() -> ChatAgent:
    """Create the finance assistant agent."""

    return ChatAgent(
        chat_client=chat_client,
        name="FinanceAssistant",
        instructions=(
            "You are a helpful personal finance assistant. "
            "Help users check balances, view transactions, and manage their money safely."
        ),
        tools=[
            get_account_balance,
            get_recent_transactions,
            transfer_money,
            pay_bill,
        ],
    )

# TODO: Build the workflow with the WorkflowBuilder()


# Simulate user request
query = FinanceQuery(
    user="john@example.com",
    request="Check my checking account balance and transfer $500 to savings",
)
print(f"\n User: {query.user}")
print(f"Request: {query.request}\n")


# TODO: Initialize variables for the approval loop
# 1. Create an empty dict to store approval responses
# 2. Create a variable to store the final output


while True:
    events = await workflow.send_responses(responses) if responses else await workflow.run(query)
    responses.clear()

    # Handle approval requests
    for event in events.get_request_info_events():
        
        # TODO: Check if the event data is a FunctionApprovalRequestContent
        # If it's NOT, raise a ValueError

        # TODO: Parse and format the function call arguments for observability

        print("Approved\n")
        # TODO: Create an approval response and add it to the responses dict

    # Check for completion
    if outputs := events.get_outputs():
        output = outputs[0]
        break

print("\n Assistant Response:")
print(output)

<details>
<summary><strong>Click to reveal solution</strong></summary>

```python
from dataclasses import dataclass
from agent_framework import executor, AgentExecutorResponse, WorkflowContext, WorkflowBuilder, FunctionApprovalResponseContent, Executor, handler
from typing import Never

@dataclass
class FinanceQuery:
    user: str
    request: str

@ai_function
def get_account_balance(account_type: str) -> dict:
    """Get the current balance for an account."""
    accounts = {
        "checking": {"balance": 5240.50, "currency": "USD"},
        "savings": {"balance": 15780.25, "currency": "USD"},
        "credit": {"balance": -1205.80, "currency": "USD"},
    }
    return accounts.get(account_type, {"error": "Account not found"})


@ai_function
def get_recent_transactions(account_type: str, limit: int = 5) -> list[dict]:
    """Get recent transactions for an account."""
    transactions = {
        "checking": [
            {"date": "2025-11-06", "description": "Grocery Store", "amount": -125.40},
            {"date": "2025-11-05", "description": "Salary Deposit", "amount": 3500.00},
            {"date": "2025-11-03", "description": "Electric Bill", "amount": -89.50},
        ],
        "credit": [
            {"date": "2025-11-06", "description": "Online Shopping", "amount": -245.30},
            {"date": "2025-11-04", "description": "Restaurant", "amount": -67.80},
        ],
    }
    return transactions.get(account_type, [])[:limit]


@ai_function(approval_mode="always_require")
async def transfer_money(
    from_account: str,
    to_account: str,
    amount: float,
) -> str:
    """Transfer money between accounts."""
    
    return f"Successfully transferred ${amount:.2f} from {from_account} to {to_account}"


@ai_function(approval_mode="always_require")
async def pay_bill(
    account: str,
    payee: str,
    amount: float,
) -> str:
    """Pay a bill from an account."""
    
    return f"Successfully paid ${amount:.2f} to {payee} from {account}"

class QueryRouter(Executor):
    def __init__(self) -> None:
        super().__init__(id="query_router")

    @handler
    async def route(self, query: FinanceQuery, ctx: WorkflowContext[str]) -> None:
        """Route the finance query to the agent."""
        message = f"User request: {query.request}"
        await ctx.send_message(message)


@executor(id="conclude_workflow")
async def conclude_workflow(
    response: AgentExecutorResponse,
    ctx: WorkflowContext[Never, str],
) -> None:
    """Output the final response."""
    await ctx.yield_output(response.agent_run_response.text)

def create_finance_agent() -> ChatAgent:
    """Create the finance assistant agent."""

    return ChatAgent(
        chat_client=chat_client,
        name="FinanceAssistant",
        instructions=(
            "You are a helpful personal finance assistant. "
            "Help users check balances, view transactions, and manage their money safely."
        ),
        tools=[
            get_account_balance,
            get_recent_transactions,
            transfer_money,
            pay_bill,
        ],
    )

workflow = (
        WorkflowBuilder()
        .register_agent(create_finance_agent, name="finance_agent")
        .register_executor(lambda: QueryRouter(), name="router")
        .register_executor(lambda: conclude_workflow, name="conclude")
        .set_start_executor("router")
        .add_edge("router", "finance_agent")
        .add_edge("finance_agent", "conclude")
        .build()
    )

# Simulate user request
query = FinanceQuery(
    user="john@example.com",
    request="Check my checking account balance and transfer $500 to savings",
)

print(f"\n User: {query.user}")
print(f"Request: {query.request}\n")

responses: dict[str, FunctionApprovalResponseContent] = {}
output: list[ChatMessage] | None = None

while True:
    events = await workflow.send_responses(responses) if responses else await workflow.run(query)
    responses.clear()

    # Handle approval requests
    for event in events.get_request_info_events():
        if not isinstance(event.data, FunctionApprovalRequestContent):
            raise ValueError(f"Unexpected request type: {type(event.data)}")

        args = json.dumps(event.data.function_call.parse_arguments(), indent=2)
        print(f"\n Approval Required: {event.data.function_call.name}")
        print(f"Arguments:\n{args}")
        
        # Auto-approve for demonstration purposes
        print("Approved\n")
        responses[event.request_id] = event.data.create_response(approved=True)

    # Check for completion
    if outputs := events.get_outputs():
        output = outputs[0]
        break

print("\n Assistant Response:")
print(output)

```

</details>

---
## Summary & Recap

Congratulations! You've learned how to implement human-in-the-loop approvals in Agent Framework.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Approval Modes** | `@ai_function(approval_mode="always_require")` for sensitive operations; others execute automatically |
| **Request-Response Pattern** | `FunctionApprovalRequestContent` (agent asks) → `FunctionApprovalResponseContent` (user decides) |
| **Conversation Flow** | Check `result.user_input_requests`, create response with `.create_response(approved)`, resume with full history |
| **Content Types** | `TextContent` (plain text), `FunctionCallContent` (tool invocation), `FunctionResultContent` (tool output), `FunctionApprovalRequestContent` (approval request), `FunctionApprovalResponseContent` (approval decision) |


You now have the skills to build:
- Human-in-the-loop approval patterns for specific operations
- Function tools with `approval_mode="always_require"`
- Interactive approval loops for production systems
- Conversation history overview and management

In **Section 4.2**, you'll learn about **advanced HITL patterns in Workflows** to give you full control of agent actions!